<a href="https://colab.research.google.com/github/ghimirebimal/ML-Projects/blob/main/Explainable_AI_using_SHAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 10: Explainable AI
### Using SHAP library to visualize and explain the decision made by the ML model

### Learning Objectives
* Learn to use SHAP(Shapely Additive exPlanations) library and force plot to visualize the features most responsible for the label.
* Learn to use catboost library to convert categorical features into numeric.
* Use the pyplot to select the few vital features responsible for the label from the trivial many.

### Install SHAP and Catboost
Install SHAP(Shapely Additive exPlanations) is a package which helps explain the predictions made by ML models using game theoretic approach. Catbooost is a package which helps utilize the categorical data directly into the model. 

In [ ]:
!pip install shap

In [ ]:
!pip3 install catboost

### Imports
Import all the necessary libraries for the lab including Pandas, numpy, matplotlib, CatBoost.

In [ ]:
import pandas as pd 
import numpy as np 

import shap
shap.initjs()
from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt

### Mount Google Drive
In the code cell below, we mount the google drive to the colab environment so that we have access to the local version of the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Read CSV
We read the csv file using pandas in the code below.

In [ ]:
mydata = pd.read_csv('/content/drive/My Drive/Intro2MLDatasets/Lab11/train.csv')

In [ ]:
mydata

### Drop Features
The features like 'Name', 'PassengerId', 'Cabin', 'Ticket' are unique and contribute no value towards the model learning. Hence, these features are dropped from the dataset.

In [ ]:
mydata.drop(columns=['Ticket','Cabin','Name','PassengerId'],inplace=True)
mydata.rename({'Sex': 'Gender'}, axis=1, inplace=True)
mydata

### Convert Age into numeric
The feature 'Age' is converted into numeric format.

In [ ]:
mydata['Age'] = pd.to_numeric(mydata['Age'])

### Categorize into Numerical and Categorical 
The features in the dataset are categorized into numerical and categorical features.

In [ ]:
def categorize(data):
  num_columns = []
  cat_columns = []

  for col in data.columns.values:
      if data[col].dtypes == 'int64' or data[col].dtypes == 'float64':
          num_columns += [col]
      else:
          cat_columns += [col]
  return [cat_columns, num_columns]

In [ ]:
cat_columns, num_columns = categorize(mydata)[0], categorize(mydata)[1]
print("Categorical values: ", cat_columns)
print("Numerical values: ", num_columns)

### Median value
For each of the numerical feature, find the median value and save it as median value of the feature.

In [ ]:
median_val = pd.Series()
for col in num_columns:
  median_val[col] = mydata[col].median()
print("Median values for each Numerical features \n\n", median_val)

### Handle missing values
As features have missing data in it, these missing values should be replaced before we can train the model. These missing values are replaced by median value generatead in the previous code for numerical features. Missing values for categorical features should be replaced by "Missing value".

In [ ]:
def handle_missing_values(data, median_val):
    df = data.copy()
    for col in df:
        if col in median_val.index.values:
            df[col] = df[col].fillna(median_val[col])
        else:
            df[col] = df[col].fillna("Missing value")
    
    return df

In [ ]:
mydata = handle_missing_values(mydata, median_val)

In [ ]:
mydata

### Label and features
Separate the dataset into label and features to prepare for training.

In [ ]:
X = mydata.drop('Survived', axis=1)
y = mydata['Survived']

### CatBoost Classifier
We are using CatBoost classifier to train the model and generate SHAP values for each feature.

In [ ]:
#State and Area Code as categorical features
categorical_features = ['Gender','Embarked']
cat = CatBoostClassifier(cat_features=categorical_features).fit(X,y)

#Use Pool to identify categorical features in X dataframe, and identify the return type of catboost library as ShapValues. 
shap_values = pd.DataFrame(
    cat.get_feature_importance(data = Pool(X, cat_features = categorical_features), type='ShapValues')[:,:-1],
    columns = X.columns,
    index = X.index
)

In [ ]:
shap_values

### Assessing the most risked factors for each customer
The plot below shows for each passenger relatively how many features are affecting the probability of survival. In most of the passengers, it looks like the number of features highly affecting the probability of survival is very close. In nature it is often found that the greatest part of an outcome is due to a tiny number of causes, and is also called pareto principle.

In [ ]:
%matplotlib inline 

#for loop through each of the rows and sort SHAP values for each row
for i in shap_values.index:
  plt.plot(range(shap_values.shape[1]), shap_values.iloc[i, :].sort_values())

plt.title('All risky customers')
plt.ylabel('SHAP')
plt.xlabel('Features sorted by SHAP (for each customer)')

#As the SHAP values are sorted, the x-axis label will be different for each customer.

### Individual Instance
This code cell picks the third instance from the dataset to visualize how each feature plays the importance to it's respective label. 

In [ ]:
X_obs1 = X.loc[3,:]
X_obs1 = X_obs1.to_frame().T
X_obs1.index = ['Observed']
X_obs1

,Pclass,Gender,Age,SibSp,Parch,Fare,Embarked
Observed,1,female,35,1,0,53.1,S


In [ ]:
X_obs = X.loc[55,:]
X_obs = X_obs.to_frame().T
X_obs.index = ['Observed']
X_obs

### Visualizing SHAP value for the instance 
The SHAP value for the third instance of the dataset is visualized where positive value has more influence on the label and negative value has less infleunce on the label.

In [ ]:
shap_obs1 = shap_values.loc[3,:]
shap_obs1 = shap_obs1.to_frame().T.round(3).astype(str)
shap_obs1.index = ['SHAP']
shap_obs1 = shap_obs1.style.apply(lambda x:["background:pink" if v[0]!='-' else "background:lightblue" for v in x], axis = 1)
shap_obs1 

In [ ]:
shap_obs = shap_values.loc[55,:]
shap_obs = shap_obs.to_frame().T.round(3).astype(str)
shap_obs.index = ['SHAP']
shap_obs = shap_obs.style.apply(lambda x:["background:pink" if v[0]!='-' else "background:lightblue" for v in x], axis = 1)
shap_obs 

### Analyzing force plot
The force plot provides the SHAP values which basically tell us which features are most likely to affect the label. Pink colored (positive values) part dictates the features which have more effect on the label. As we go from left to right, the probability of label being true is higher. Blue colored (negative values) part dictates the features which have less effect on label. As we go from right to left, the probability of label being false is higher.

In [ ]:
shap.force_plot(
    base_value = 0, 
    shap_values = np.array(shap_values.loc[3,:]), 
    features = X.loc[3,:],
    show = False,
    matplotlib=True
)
plt.tight_layout()

In [ ]:
shap.force_plot(
    base_value = 0, 
    shap_values = np.array(shap_values.loc[55,:]), 
    features = X.loc[55,:],
    show = False,
    matplotlib=True
)
plt.tight_layout()

### Risk factors in single passenger
Here we visualize the features that highly affect the label for a single passenger.

In [ ]:
%matplotlib inline

SHAP_values1 = shap_values.iloc[3,:].sort_values()
plt.plot(range(shap_values.shape[1]), SHAP_values1, '-bo')
plt.hlines(0, -.5, shap_values.shape[1] - .5, color = 'red')
plt.ylabel('SHAP')
plt.xlabel('Features sorted by SHAP')
plt.title('High-risk passenger (id: {})'.format(3))
plt.xticks(range(shap_values.shape[1]), SHAP_values1.index, rotation=90)

In [ ]:
X_obs1

In [ ]:
y_obs1 = y.iloc[3]
print("Survived: ", y_obs1)

In [ ]:
%matplotlib inline

SHAP_values = shap_values.iloc[55,:].sort_values()
plt.plot(range(shap_values.shape[1]), SHAP_values, '-bo')
plt.hlines(0, -.5, shap_values.shape[1] - .5, color = 'red')
plt.ylabel('SHAP')
plt.xlabel('Features sorted by SHAP')
plt.title('High-risk passenger (id: {})'.format(55))
plt.xticks(range(shap_values.shape[1]), SHAP_values.index, rotation=90)

In [ ]:
X_obs

In [ ]:
y_obs = y.iloc[55]
print("Survived: ", y_obs)

### Cumulative sum of the SHAP values
As we see in the plot below, the vertical line separates the negative and positive cumulative SHAP values of the features. Cumulative sum helps us separate vital few causes from the trivial many. As we can see in the plot, four major features seem to be directly affecting the label negatively in this case.

In [ ]:
%matplotlib inline 

cum_SHAP_values1 = SHAP_values1.cumsum()
cum_range = max(cum_SHAP_values1) - min(cum_SHAP_values1)

plt.plot(range(shap_values.shape[1]), cum_SHAP_values1, '-bo')
plt.vlines(shap_values.shape[1] - (cum_SHAP_values1 > 0).sum() - .5, min(cum_SHAP_values1) - .05 * cum_range, max(cum_SHAP_values1) + .05 * cum_range, color = 'red')
plt.hlines(0, -.5, shap_values.shape[1] - .5, color = 'red')
plt.ylabel('SHAP cumulative sum')
plt.xlabel('Features sorted by SHAP')
plt.title('High-risk passenger (id: {})'.format(3))
plt.xticks(range(shap_values.shape[1]), cum_SHAP_values1.index, rotation=90)

In [ ]:
X_obs1

In [ ]:
%matplotlib inline 

cum_SHAP_values = SHAP_values.cumsum()
cum_range = max(cum_SHAP_values) - min(cum_SHAP_values)

plt.plot(range(shap_values.shape[1]), cum_SHAP_values, '-bo')
plt.vlines(shap_values.shape[1] - (cum_SHAP_values > 0).sum() - .5, min(cum_SHAP_values) - .05 * cum_range, max(cum_SHAP_values) + .05 * cum_range, color = 'red')
plt.hlines(0, -.5, shap_values.shape[1] - .5, color = 'red')
plt.ylabel('SHAP cumulative sum')
plt.xlabel('Features sorted by SHAP')
plt.title('High-risk passenger (id: {})'.format(55))
plt.xticks(range(shap_values.shape[1]), cum_SHAP_values.index, rotation=90)

In [ ]:
X_obs

**Can you show a comparison of force plot for different instance of Female and Male passenger? Explain what you observed.**